In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from nameparser import HumanName

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
brig = pd.read_csv('Beth_Israel_Deaconess_Medical_Center_2020-01-02.csv')

In [3]:
len(brig)

2999

In [12]:
ppd = pd.read_csv('ppd_spec.csv')

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (14,27,28,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# def clean_name(name):
#     name_split = name.split(',')
#     full_name = name_split[0].upper()
#     full_name_split = full_name.split(' ')
#     first_name = full_name_split[0]
# #     print(first_name)
#     last_name = full_name_split[-1]
# #     print(last_name)
#     full_name_split.remove(first_name)
#     middle_name = 'None'
#     if len(full_name_split) > 0:
#         full_name_split.remove(last_name)
#         if len(full_name_split)>0:
#             middle_name = full_name_split[0]
#             for word in full_name_split[1:]:
#                 middle_name = middle_name + ' ' + word
#     name_dict = {
#         'NAME':name,
#         'FIRST_NAME': first_name,
#         'LAST_NAME': last_name,
#         'MIDDLE_NAME': middle_name
#     }
#     return(name_dict)

def split_names(df):
    df = df.drop_duplicates()
    dict_list = []
    for row in df.itertuples():
        name_parsed = HumanName(row.NAME)
        name_dict = {
        'NAME':row.NAME,
        'FIRST_NAME': name_parsed.first.upper(),
        'LAST_NAME': name_parsed.last.upper(),
        'MIDDLE_NAME': name_parsed.middle.upper(),
        'SUFFIX':name_parsed.suffix.upper(),
        'NICKNAME':name_parsed.nickname.upper(),
        'TITLE':name_parsed.title.upper(),
        }
        dict_list.append(name_dict)
    name_df = pd.DataFrame(dict_list)
    new_df = pd.merge(name_df, df, on = 'NAME')
    return(new_df)

In [5]:
def match_ppd_name(ppd,df):
    count = len(df)
    matched = pd.merge(ppd, df, on = ['FIRST_NAME','LAST_NAME'], how = 'right')
    names = []
    duplicate_names = []
    missing_names =[]
    for row in matched.itertuples():
        if row.NAME not in names:
            names.append(row.NAME)
        elif row.NAME not in duplicate_names:
            duplicate_names.append(row.NAME)
        if pd.isna(row.ME)==True:
            missing_names.append(row.NAME)
    
    duplicates = matched[matched.NAME.isin(duplicate_names)]
    matched = matched.drop_duplicates('NAME', keep = False)
    
    print(f'{len(duplicates)} duplicates found.')
    print(f'{len(missing_names)} names missing.')
    
    return(duplicates, missing_names, matched)

In [6]:
def count_matches(matched_df):
    names = []
    duplicate_names = []
    for row in matched_df.itertuples():
        if row.NAME not in names:
            names.append(row.NAME)
        elif row.NAME not in duplicate_names:
            duplicate_names.append(row.NAME)
    
    duplicates = matched_df[matched_df.NAME.isin(duplicate_names)]
    matched = matched_df.drop_duplicates('NAME', keep = False)
    
    print(f'{len(duplicates)} duplicates found.')
    print(f'{len(matched)} unique matches found.')
    
    return(duplicates, matched)

In [7]:
def match_ppd_spec_fuzzy(matched_df):
    keep_list = []
    for row in matched_df.itertuples():
        keep = False
        if fuzz.ratio(row.SPECIALTY.upper(), row.DESC_PRIM) > 80:
            keep = True
        elif row.DESC_PRIM in row.SPECIALTY.upper():
            keep = True
        elif row.SPECIALTY.upper() in row.DESC_PRIM:
            keep = True
        keep_list.append(keep)
    matched_df['SPEC_MATCH']=keep_list
    matched_df = matched_df[matched_df['SPEC_MATCH']==True]
    return(matched_df)

In [8]:
def match_ppd_state(matched_df):
    state_match_list = []
    for row in matched_df.itertuples():
        state_match = False
        if row.POLO_STATE == row.LOCATION_ADDRESS_STATE:
            state_match = True
        state_match_list.append(state_match)
    matched_df['STATE_MATCH']=state_match_list
    matched_df = matched_df[matched_df['STATE_MATCH']==True]
    return(matched_df)
        
def match_ppd_city(matched_df):
    city_match_list = []
    for row in matched_df.itertuples():
        city_match = False
        if row.POLO_CITY == row.LOCATION_ADDRESS_CITY.upper():
            city_match = True
        city_match_list.append(city_match)
    matched_df['CITY_MATCH']=city_match_list
    matched_df = matched_df[matched_df['CITY_MATCH']==True]
    return(matched_df)

def match_ppd_middle(matched_df):
    middle_match_list =[]
    for row in matched_df.itertuples():
        mid_match = False
        if row.MIDDLE_NAME_y != 'None':
            if row.MIDDLE_NAME_x == row.MIDDLE_NAME_y:
                mid_match = True
            elif row.MIDDLE_NAME_x[0] == row.MIDDLE_NAME_y[0]:
                mid_match = True
        middle_match_list.append(mid_match)
    matched_df['MID_MATCH']=middle_match_list
    matched_df = matched_df[matched_df['MID_MATCH']==True]
    return(matched_df)

def match_ppd_spec_hard(matched_df):
    spec_match_list = []
    for row in matched_df.itertuples():
        spec_match = False
        if row.DESC_PRIM == row.SPECIALTY.upper():
            spec_match = True
        spec_match_list.append(spec_match)
    matched_df['SPEC_MATCH']=spec_match_list
    matched_df = matched_df[matched_df['SPEC_MATCH']==True]
    return(matched_df)

In [9]:
brig.drop_duplicates('NAME')

,NAME,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_STREET_ADDRESS,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME
0,"Amtul Aala, MD",Nephrology,None,16176329700,16176329820,Beth Israel Deaconess Medical Center,110 Francis Street,"The Transplant Center, 110 Francis Street, 7th...",Boston,MA,02215-5501,None,None,Beth Israel Deaconess Medical Center
1,"Bassima J Abdallah, MD",Internal Medicine,None,16173131377,16177548632,Beth Israel Deaconess Hospital - Milton Hospit...,199 Reedsdale Road,199 Reedsdale Road,Milton,MA,02186-3926,None,None,Beth Israel Deaconess Medical Center
2,"Ayesha Abdeen, MD",Orthopaedic Surgery,None,16176673940,16176672155,Beth Israel Deaconess Hospital - Needham,148 Chestnut Street,"Outpatient Clinical Center, 148 Chestnut Stree...",Needham,MA,2492,None,None,Beth Israel Deaconess Medical Center
3,"Dorina R. Abdulah, MD",Internal Medicine,None,1617754747,1617522903,BIDHC Extended Care Community Program,464 Hillside Avenue,"464 Hillside Avenue, Suite 304",Needham,MA,02494-1228,None,None,Beth Israel Deaconess Medical Center
4,"Charbel M. Abou Rjeily, MD",Internal Medicine,None,15087461259,15087476685,Beth Israel Deaconess HealthCare - Plymouth,275 Sandwich Street,275 Sandwich Street,Plymouth,MA,02360-2183,None,None,Beth Israel Deaconess Medical Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2994,"Dimitar V. Zlatev, MD",Urology,None,16176653590,16176652825,CHA - Urology - Cambridge Campus,1493 Cambridge Street,1493 Cambridge Street,Cambridge,MA,02139-1047,None,None,Beth Israel Deaconess Medical Center
2995,"Kate P Zona, Ph.D",Psychology,None,1781338500,17813380,CHA - Malden Family Medicine Center,195 Canal Street,195 Canal Street,Malden,MA,02148-6701,None,None,Beth Israel Deaconess Medical Center
2996,"Kangni Zong Grob, MD",Internal Medicine,None,16176651000,16176651671,CHA - Hospitalist Service - Cambridge Campus,1493 Cambridge Street,1493 Cambridge Street,Cambridge,MA,02139-1047,None,None,Beth Israel Deaconess Medical Center
2997,"Elizabeth Zulinska, MD",Cytopathology,None,16176651220,16176651205,CHA - Pathology Department,1493 Cambridge Street,1493 Cambridge Street,Cambridge,MA,02139-1047,None,None,Beth Israel Deaconess Medical Center


In [9]:
brig_df = split_names(brig)
brig_df = brig_df.drop_duplicates('NAME')

In [11]:
brig_df.head()

,NAME,FIRST_NAME,LAST_NAME,MIDDLE_NAME,SUFFIX,NICKNAME,TITLE,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_STREET_ADDRESS,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME
0,"Amtul Aala, MD",AMTUL,AALA,,MD,,,Nephrology,None,16176329700,16176329820,Beth Israel Deaconess Medical Center,110 Francis Street,"The Transplant Center, 110 Francis Street, 7th...",Boston,MA,02215-5501,None,None,Beth Israel Deaconess Medical Center
1,"Bassima J Abdallah, MD",BASSIMA,ABDALLAH,J,MD,,,Internal Medicine,None,16173131377,16177548632,Beth Israel Deaconess Hospital - Milton Hospit...,199 Reedsdale Road,199 Reedsdale Road,Milton,MA,02186-3926,None,None,Beth Israel Deaconess Medical Center
2,"Ayesha Abdeen, MD",AYESHA,ABDEEN,,MD,,,Orthopaedic Surgery,None,16176673940,16176672155,Beth Israel Deaconess Hospital - Needham,148 Chestnut Street,"Outpatient Clinical Center, 148 Chestnut Stree...",Needham,MA,2492,None,None,Beth Israel Deaconess Medical Center
3,"Dorina R. Abdulah, MD",DORINA,ABDULAH,R.,MD,,,Internal Medicine,None,1617754747,1617522903,BIDHC Extended Care Community Program,464 Hillside Avenue,"464 Hillside Avenue, Suite 304",Needham,MA,02494-1228,None,None,Beth Israel Deaconess Medical Center
4,"Charbel M. Abou Rjeily, MD",CHARBEL,RJEILY,M. ABOU,MD,,,Internal Medicine,None,15087461259,15087476685,Beth Israel Deaconess HealthCare - Plymouth,275 Sandwich Street,275 Sandwich Street,Plymouth,MA,02360-2183,None,None,Beth Israel Deaconess Medical Center


In [10]:
specs = list(brig_df['SPECIALTY'].unique())
bad_spec = ['Physician Assistant',
 'Nurse Practitioner',
 'Anesthesia - CRNA',
 'Midwife',
 'Anesthesia - CRNA, Nurse Practitioner',
 'Chiropractor',
 'Psychologist',
 'Optometrist',
 'Pediatric Dentistry',
 'Veterinarian',
 'Midwife, Nurse Practitioner']
specs = [x for x in specs if x not in bad_spec]

brig_df = brig_df[brig_df['SPECIALTY'].isin(specs)]

In [14]:
len(brig_df)

2854

In [22]:
matched_brig.drop_duplicates()

,BIRTH_CITY,BIRTH_COUNTRY,BIRTH_STATE,BIRTH_YEAR,CITY,CITY_MATCH,DEPARTMENT,DESC_PRIM,DESC_SEC,FAX,...,STATE,STATE_MATCH,SUFFIX_x,SUFFIX_y,TELEPHONE_NUMBER,TITLE,TOP_CD,TRAINING_TYPE,YEAR_IN_PROGRAM,ZIP
0,EVANSTON,US1,IL,1965.0,CAMBRIDGE,NaN,None,NEUROLOGY,NEUROMUSCULAR MEDICINE (NEUROLOGY),16176652777,...,MA,NaN,NaN,MD,6.176651e+09,,20.0,NaN,NaN,02139
68,INDIA,NaN,NaN,1961.0,BOSTON,NaN,None,NEUROLOGY,NEUROMUSCULAR MEDICINE (NEUROLOGY),16176673175,...,MA,NaN,NaN,MD,NaN,,20.0,NaN,NaN,02215
69,WASHINGTON,US1,DC,1961.0,LOWELL,NaN,Anesthesia,ANESTHESIOLOGY,PAIN MANAGEMENT,17814537350,...,MA,NaN,NaN,MD,7.814078e+09,,20.0,NaN,NaN,01852
114,PROVIDENCE,US1,RI,1961.0,COHASSET,NaN,"Anesthesia, Critical Care & Pain Med",ANESTHESIOLOGY,PAIN MANAGEMENT,16176677849,...,MA,NaN,NaN,MD,8.006675e+09,,20.0,NaN,NaN,2025.0
124,CLEVELAND,US1,OH,1960.0,WELLESLEY,NaN,Anesthesia,ANESTHESIOLOGY,PAIN MANAGEMENT,17814537350,...,MA,NaN,NaN,MD,7.814078e+09,,20.0,NaN,NaN,02482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6048,NaN,EG1,NaN,1981.0,BOSTON,NaN,None,INTERNAL MEDICINE,UNSPECIFIED,16172473912,...,MA,True,NaN,MD,NaN,,20.0,NaN,NaN,02118
1326,NEW YORK,US1,NY,1938.0,BOSTON,True,None,NEPHROLOGY,INTERNAL MEDICINE,16176329890,...,MA,True,NaN,MD,6.176330e+09,,40.0,NaN,NaN,02215
3606,BOSTON,US1,MA,1968.0,CHESTNUT HILL,True,None,INTERNAL MEDICINE,UNSPECIFIED,1617731825,...,MA,True,NaN,MD,6.177310e+09,,20.0,NaN,NaN,02467
5272,GLENS FALLS,US1,NY,1960.0,NEEDHAM,True,None,INTERNAL MEDICINE,UNSPECIFIED,17814441907,...,MA,True,NaN,"MD, MPH",7.814556e+09,,20.0,NaN,NaN,02492


In [13]:
duplicate_names_brig, missing_names_brig, matched_brig = match_ppd_name(ppd, brig_df)

4825 duplicates found.
277 names missing.


In [61]:
new_matched = match_ppd_spec_fuzzy(duplicate_names_brig)
new_matched.head()
new_duplicates, new_matches = count_matches(new_matched)
new_matches.head()
print(len(matched_brig))
matched_brig = matched_brig.append(new_matches)
print(len(matched_brig))
print(f'{len(new_matches)} new matches confirmed. {len(matched_brig)} total matches confirmed.')

new_matched_2 = match_ppd_state(new_duplicates)
new_duplicates_2, new_matches_2 = count_matches(new_matched_2)
print(len(matched_brig))
matched_brig = matched_brig.append(new_matches_2)
print(len(matched_brig))
print(f'{len(new_matches_2)} new matches confirmed. {len(matched_brig)} total matches confirmed.')

new_matched_3 = match_ppd_city(new_duplicates_2)
new_duplicates_3, new_matches_3 = count_matches(new_matched_3)
matched_brig = matched_brig.append(new_matches_3)
print(f'{len(new_matches_3)} new matches confirmed. {len(matched_brig)} total matches confirmed.')

new_matched_4 = match_ppd_middle(new_duplicates_3)
new_duplicates_4, new_matches_4 = count_matches(new_matched_4)
matched_brig = matched_brig.append(new_matches_4)
print(f'{len(new_matches_4)} new matches confirmed. {len(matched_brig)} total matches confirmed.')

new_matched_5 = match_ppd_spec_hard(new_duplicates_4)
new_duplicates_5, new_matches_5 = count_matches(new_matched_5)
matched_brig = matched_brig.append(new_matches_5)
print(f'{len(new_matches_5)} new matches confirmed. {len(matched_brig)} total matches confirmed.')

315 duplicates found.
417 unique matches found.
2212
2629
417 new matches confirmed. 2629 total matches confirmed.
16 duplicates found.
87 unique matches found.
2629
2716
87 new matches confirmed. 2716 total matches confirmed.
6 duplicates found.
3 unique matches found.
3 new matches confirmed. 2719 total matches confirmed.
0 duplicates found.
3 unique matches found.
3 new matches confirmed. 2722 total matches confirmed.

C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\vigrose\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value


0 duplicates found.
0 unique matches found.
0 new matches confirmed. 2722 total matches confirmed.


In [62]:
matched_brig

,BIRTH_CITY,BIRTH_COUNTRY,BIRTH_STATE,BIRTH_YEAR,CITY,CITY_MATCH,DEPARTMENT,DESC_PRIM,DESC_SEC,FAX,...,STATE,STATE_MATCH,SUFFIX_x,SUFFIX_y,TELEPHONE_NUMBER,TITLE,TOP_CD,TRAINING_TYPE,YEAR_IN_PROGRAM,ZIP
0,EVANSTON,US1,IL,1965.0,CAMBRIDGE,NaN,None,NEUROLOGY,NEUROMUSCULAR MEDICINE (NEUROLOGY),16176652777,...,MA,NaN,NaN,MD,6.176651e+09,,20.0,NaN,NaN,02139
68,INDIA,NaN,NaN,1961.0,BOSTON,NaN,None,NEUROLOGY,NEUROMUSCULAR MEDICINE (NEUROLOGY),16176673175,...,MA,NaN,NaN,MD,NaN,,20.0,NaN,NaN,02215
69,WASHINGTON,US1,DC,1961.0,LOWELL,NaN,Anesthesia,ANESTHESIOLOGY,PAIN MANAGEMENT,17814537350,...,MA,NaN,NaN,MD,7.814078e+09,,20.0,NaN,NaN,01852
114,PROVIDENCE,US1,RI,1961.0,COHASSET,NaN,"Anesthesia, Critical Care & Pain Med",ANESTHESIOLOGY,PAIN MANAGEMENT,16176677849,...,MA,NaN,NaN,MD,8.006675e+09,,20.0,NaN,NaN,2025.0
124,CLEVELAND,US1,OH,1960.0,WELLESLEY,NaN,Anesthesia,ANESTHESIOLOGY,PAIN MANAGEMENT,17814537350,...,MA,NaN,NaN,MD,7.814078e+09,,20.0,NaN,NaN,02482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3606,BOSTON,US1,MA,1968.0,CHESTNUT HILL,True,None,INTERNAL MEDICINE,UNSPECIFIED,1617731825,...,MA,True,NaN,MD,6.177310e+09,,20.0,NaN,NaN,02467
5272,GLENS FALLS,US1,NY,1960.0,NEEDHAM,True,None,INTERNAL MEDICINE,UNSPECIFIED,17814441907,...,MA,True,NaN,"MD, MPH",7.814556e+09,,20.0,NaN,NaN,02492
4040,ROCHESTER,US1,NY,1973.0,NEWTON CENTRE,True,None,ADDICTION PSYCHIATRY,UNSPECIFIED,16179755274,...,MA,True,NaN,"MD, MHS",6.176672e+09,,20.0,NaN,NaN,02459
4240,NaN,NaN,NaN,1968.0,QUINCY,True,None,FAMILY MEDICINE,UNSPECIFIED,1617754230,...,MA,True,NaN,MD,6.177540e+09,,20.0,NaN,NaN,02171


In [28]:
missing_df = henry_df[henry_df['NAME'].isin(missing_names_henry)]

In [29]:
add_matches = []
for name in missing_names_henry:
    if name != 'None':
        first_match = ppd[ppd['FIRST_NAME'] == name.upper().split(' ')[0]]
        last_match = ppd[ppd['LAST_NAME'] == name.upper().split(' ')[1]]
        if len(first_match) == 1:
            print(f'{name} : {list(first_match["MAILING_NAME"])[0]}')
            new_dict ={'MAILING_NAME': list(first_match['MAILING_NAME'])[0],
                       'ME': list(first_match['ME'])[0],
                       'DESC_PRIM': list(first_match['DESC_PRIM'])[0],
                       'NAME':name
            }
            add_matches.append(new_dict)
        if len(last_match) == 1:
            print(f'{name} : {list(last_match["MAILING_NAME"])[0]}')
            new_dict ={'MAILING_NAME': list(last_match['MAILING_NAME'])[0],
                       'ME': list(last_match['ME'])[0],
                       'DESC_PRIM': list(last_match['DESC_PRIM'])[0],
                       'NAME':name
            }
            add_matches.append(new_dict)

Muwaffak Abdulhak : MUWAFFAK M ABDUL-HAK MD
Odaliz Abreu Lanfranco : ODALIZ E ABREU LANFRANCO MD
Kheir Al-Zouhayli : M K AL-ZOUHAYLI MD
Salah Alenzi : BADER A ALENZI MD
Wamidh Alkhoory : WAMIDH L ALKHURRI MD
Lisa Allenspach : ERIC J ALLENSPACH MD
Mohamed-Rida Alsaden : MUHAMADRIDA A ALSADEN MD
Karthikeyan Ananthasubramaniam : K ANANTHASUBRAMANIAM MD
Graham Appleford : COLIN APPLEFORD DO
Samah Arsanious : DAVID E ARSANIOUS MD
Gregory Auner : JOHN D AUNER MD
Fatima Awarke : FATME AWARKE DO
Brian Bismack : GREGORY T BISMACK MD
Abby Bonato : ALEXANDER E BONATO MD
Jieli Chen : JIELI LI MD
Laura Dalla Vecchia : KUMAR P DALLA MD
Sairia Dass Ramcharan : SAIRIA DASS MD
Janice Demaray : MICHAEL J DEMARAY MD
Elaine Dziemianowicz : MARK S DZIEMIANOWICZ MD
Samar El-Achkar : TAREK M EL-ACHKAR MD
Stephanie Franso : DEANE FRANSO MD
Stephen Frascone : RALPH J FRASCONE MD
Shira Gordinier : MARY E GORDINIER MD
Gurpriya Gupta : GURPRIYA K SETHI MD
Xiaoxia Han : XIAOXIA ZHANG MD
Paul Heilborn : PAUL-DAVID 

KeyboardInterrupt: 

In [ ]:
attempts = pd.DataFrame(add_matches)
TEST = pd.merge(attempts, henry_df, on = 'NAME')
scores =[]
for row in TEST.itertuples():
    score =fuzz.ratio(row.MAILING_NAME,row.NAME.upper())
    scores.append(score)
TEST['SCORE']=scores

In [ ]:
TEST[TEST['SCORE']>72]

In [ ]:
first_df.to_csv('inf_test.csv')

In [ ]:
for doc in missing_names_henry:
    matched = False
    for NAME in names:
        if fuzz.ratio(NAME, doc.upper())>70:
            potential_list.append(NAME)
            print(f'{NAME} : {doc}')
            matched = True
            if fuzz.ratio(NAME, doc.upper())>84:
                break
    if matched == False:
        print(f'{doc} not found')

In [ ]:
names = list(ppd['MAILING_NAME'])

In [ ]:
fuzz.ratio('MUWAFFAK M ABDUL-HAK MD','MUWAFFAK ABDULHAK')

In [ ]:
missing_names_henry[9:].sort()

In [ ]:
missing_names_henry.sort()

In [ ]:
names.sort()

In [ ]:
pd.merge(first_df, henry_df, on='LAST_NAME')[['NAME','MAILING_NAME']].head(50)

In [ ]:
ppd.dtypes

In [ ]:
missing_df.groupby('SPECIALTY').count().sort_values(by = 'NAME',ascending = False).head(50)

In [ ]:
missing = missing_df[missing_df['SPECIALTY'].isin(specs)]

In [ ]:
specs.remove(bad_spec)

In [ ]:
specs = [x for x in specs if x not in bad_spec]

In [ ]:
def look_for_doctor(name):
    potential_list =[]
    name_scores = []
    matched = False
    for NAME in names:
        score = fuzz.ratio(NAME, name.upper())
        if score > 70:
            potential_list.append(NAME)
            name_scores.append(score)
            print(f'{NAME} : {name}, {score}')
            matched = True
            if score >84:
                break
    if matched == False:
        print(f'{name} not found')
    return(potential_list, name_scores)

def match_doctor(name, specialty):
    potential_list, name_scores = look_for_doctor(name)
    if len(potential_list)==0:
        return([])
    filtered_ppd = ppd[ppd['MAILING_NAME'].isin(potential_list)]
    spec_scores = []
    for row in filtered_ppd.itertuples():
        spec_score = fuzz.ratio(row.DESC_PRIM, specialty.upper())
        spec_scores.append(spec_score)
    filtered_ppd['SPEC_SCORE']=spec_scores
    filtered_ppd['NAME_SCORE']=name_scores
    filtered_ppd['SCORE']=filtered_ppd['SPEC_SCORE']+filtered_ppd['NAME_SCORE']
    return(filtered_ppd)

def pick_match(name, specialty):
    filtered_ppd = match_doctor(name, specialty)
    if len(filtered_ppd)==0:
        return([])
    return(filtered_ppd[filtered_ppd.SPEC_SCORE == filtered_ppd.SPEC_SCORE.max()])

In [ ]:
potential_finds = pick_match('JOELLE ABOOD', 'Obstetrics & Gynecology')

In [ ]:
potential_finds = ppd[ppd['MAILING_NAME']=='JOELLE AOUN MD']

In [ ]:
j_list = ['JOEL E ABBOTT DO',
 'JOEL E ARROYO MD',
 'JOEL E BORKOW MD',
 'JOEL E BROOK MD',
 'JOELLE AOUN MD',
 'JOELLE MORROW MD',
 'JOELLEN BROWN MD',
 'NOELLE BOCTOR MD']
for thing in j_list:
    print(thing)
    print(fuzz.ratio('JOELLE ABOOD',thing))
    print('')

In [ ]:
fuzz.ratio('JOELLE ABOOD', 'JOELLE MORROW MD')

In [ ]:
ppd = ppd.sort_values('MAILING_NAME')

In [ ]:
num = 144
for row in missing[144:].itertuples():
    print(num)
    matches = pick_match(row.NAME, row.SPECIALTY)
    if len(matches)!=0:
        potential_finds.append(matches)
    num += 1

In [ ]:
missing[144:150]

In [ ]:
dict_list =[]
duplicate_df = henry_df[henry_df['NAME'].isin(duplicate_names_henry)]
duplicate_complete = pd.merge(duplicate_df, ppd, on=['FIRST_NAME','LAST_NAME'])
for row in duplicate_complete.itertuples():
    if row.SPECIALTY in specs:
        print(f'{row.NAME} : {row.MAILING_NAME}\n{row.SPECIALTY} : {row.DESC_PRIM}')
        print('')
        keep = False
        if fuzz.ratio(row.SPECIALTY.upper(), row.DESC_PRIM) > 80:
            keep = True
        elif row.DESC_PRIM in row.SPECIALTY.upper():
            keep = True
        elif row.SPECIALTY.upper() in row.DESC_PRIM:
            keep = True
        if keep == True:
            new_dict = {
                    'NAME': row.NAME,
                    'MAILING_NAME':row.MAILING_NAME,
                    'ME': row.ME
                }
            dict_list.append(new_dict)

In [ ]:
duplicate_complete['DESC_PRIM']

In [ ]:
len(pd.DataFrame(dict_list).drop_duplicates('NAME',keep=False))

In [ ]:
len(henry_df[henry_df['SPECIALTY'].isin(specs)])

In [ ]:
len(matched_henry[matched_henry['SPECIALTY'].isin(specs)].dropna(subset = ['ME']).drop_duplicates('NAME',keep=False)[['DESC_PRIM','SPECIALTY']])

In [ ]:
len(missing)

In [ ]:
'226 duplicates'

In [ ]:
potential_finds

In [ ]:
spec_name_matches = pd.DataFrame(dict_list)

In [ ]:
spec_name_matches_ = pd.merge(spec_name_matches, henry_df, on='NAME')
spec_name_matches_ = pd.merge(spec_name_matches_, ppd, on='ME')

In [ ]:
for col in spec_name_matches_.columns:
    print(col)

In [ ]:
state_match_list = []
for row in spec_name_matches_.itertuples():
    state_match = False
    if row.POLO_STATE == row.LOCATION_ADDRESS_STATE:
        state_match = True
    state_match_list.append(state_match)

In [ ]:
spec_name_matches_['STATE_MATCH']=state_match_list

In [ ]:
spec_name_matches_[spec_name_matches_['STATE_MATCH']==True][['SPECIALTY','DESC_PRIM','NAME','MAILING_NAME_x', 'POLO_CITY','LOCATION_ADDRESS_CITY']][350:400]

In [ ]:
2+2

In [ ]:
matched_henry

In [ ]:
matched_henry.dropna(subset = ['ME'])

In [ ]:
len(henry_df)

In [ ]:
for col in henry_df.columns:
    print(col)

In [ ]:
all_henry = pd.merge(henry_df, matched_henry, on = ['NAME',
'FIRST_NAME',
'LAST_NAME',
'NICKNAME',
'TITLE',
'SPECIALTY',
'SECONDARY_SPECIALTY',
'PHONE',
'FAX',
'LOCATION_NAME',
'LOCATION_ADDRESS',
'LOCATION_ADDRESS_CITY',
'LOCATION_ADDRESS_STATE',
'LOCATION_ADDRESS_ZIP',
'DEPARTMENT',
'PROGRAM',
'PARENT_ORG_NAME'], how = 'left')

In [ ]:
len(henry_df)

In [ ]:
still_na = all_henry[all_henry['ME'].isna()]

In [ ]:
def match_cleaned_last(still_na,ppd):
    cleaned_lasts_2 = []
    for last in list(still_na['LAST_NAME']):
        new_last = str(last).replace(' ','').replace(' ','').replace('-','').replace("'","")
        cleaned_lasts_2.append(new_last)
    still_na['CLEANED_LAST']=cleaned_lasts_2
    cleaned_lasts = []
    for last in list(ppd['LAST_NAME']):
        new_last = str(last).replace(' ','').replace(' ','').replace('-','').replace("'","")
        cleaned_lasts.append(new_last)
    ppd['CLEANED_LAST']=cleaned_lasts
    new_matches = pd.merge(still_na,ppd,on=['FIRST_NAME','CLEANED_LAST'])
    return(new_matches)

In [ ]:
cleaned_matches = match_cleaned_last(still_na, ppd)

In [ ]:
clean_duplicates, clean_matches = count_matches(cleaned_matches)

In [ ]:
matched_henry.append(clean_matches)

In [ ]:
ppd_last_names=list(ppd['LAST_NAME'])

In [ ]:
ppd_last_names

In [ ]:
cleaned_lasts_2 = []
for last in list(still_na['LAST_NAME']):
#     if "'" in str(last):
#         print(last)
    new_last = str(last).replace(' ','').replace(' ','').replace('-','').replace("'","")
    cleaned_lasts_2.append(new_last)

In [ ]:
for row in still_na.itertuples():
    if " " in row.LAST_NAME:
        print(row.LAST_NAME)

In [ ]:
ppd['CLEANED_LAST']=cleaned_lasts

In [ ]:
still_na['CLEANED_LAST']=cleaned_lasts_2

In [ ]:
len(still_na)

In [ ]:
clean_duplicates.drop_duplicates('NAME')

In [ ]:
clean_matches

In [ ]:
still_na.drop_duplicates('NAME')

In [ ]:
missing_list = []
COUNT = 0
for row in still_na.itertuples():
    if row.NAME not in founded_names:
        COUNT += 1
        print(row.NAME)
        print(row.SPECIALTY)
        print(row.LOCATION_ADDRESS_CITY)
        print(row.LOCATION_ADDRESS_STATE)
        new_dic  = {
            'NAME':row.NAME,
            'FIRST_NAME':row.FIRST_NAME,
            'MIDDLE_NAME': row.MIDDLE_NAME,
            'LAST_NAME':row.LAST_NAME,
            'SPEC':row.SPECIALTY,
            'CITY':row.LOCATION_ADDRESS_CITY,
            'STATE':row.LOCATION_ADDRESS_STATE
        }
        missing_list.append(new_dic)
print(COUNT)
MISSING_DF = pd.DataFrame(missing_list)

In [ ]:
founded_names = list(cleaned_matches['NAME'])

In [ ]:
ppd[ppd['FIRST_NAME']=='SHANA'][0:51]

In [ ]:
ppd

In [ ]:
first_match = pd.merge(MISSING_DF, ppd, on = 'FIRST_NAME')

In [ ]:
first_match = first_match.rename(columns={'SPEC':'SPECIALTY'})

In [ ]:
spec_first = match_ppd_spec_fuzzy(first_match)

In [ ]:
spec_first_duplicates, spec_first_matches = count_matches(spec_first)

In [ ]:
for row in MISSING_DF.itertuples():
    pd.merge(MISSING_DF, ppd, on=)

In [ ]:
fuzz.ratio('LANFRANCO','ABREAU LANFRANCO')

In [ ]:
spec_first_matches[['NAME','LAST_NAME_x','LAST_NAME_y']]

In [ ]:
def match_check(thing_1, thing_2):
    match = False
    if str(thing_1).upper() == str(thing_2).upper():
        match = True
    if str(thing_2).upper() in str(thing_1).upper():
        match =True
    if fuzz.ratio(str(thing_1).upper(), str(thing_2).upper())>70:
        match =True
    return(match)

In [ ]:
def check_first_matches(MISSING_DF):
    me_name_list =[]
    for row in MISSING_DF.itertuples():
        first_match = ppd[ppd['FIRST_NAME']==row.FIRST_NAME]
        if len(first_match)==1:
            me_name = {'NAME':row.NAME,
                       'ME': list(first_match['ME'])[0]
                        }
            if match_check(list(first_match['DESC_PRIM'])[0],row.SPEC)==True:
                if match_check(list(first_match['LAST_NAME'])[0],row.LAST_NAME)==True:
                    me_name_list.append(me_name)

        else:
            for line in first_match.itertuples():
                if match_check(line.DESC_PRIM,row.SPEC)==True:
                    if match_check(line.LAST_NAME,row.LAST_NAME)==True:
                        me_name = {'NAME':row.NAME,
                       'ME': line.ME
                        }
                        me_name_list.append(me_name)
    

In [ ]:
MISSING_DF

In [ ]:
match_check('ABREU LANFRANCO','LANFRANCO')

In [ ]:
ppd[ppd['FIRST_NAME']=='ODALIZ']

In [ ]:
me_name_list

In [31]:
matched_henry.dropna(subset = ['ME']).to_csv('Matched_Henry.csv',index=False)

In [42]:
matched_henry[['PHONE','TELEPHONE_NUMBER']]

,PHONE,TELEPHONE_NUMBER
0,800-436-7936,3.139163e+09
139,586-263-2371,5.862632e+09
173,586-726-4823,5.867265e+09
214,517-205-7525,5.172051e+09
215,800-436-7936,7.349367e+09
...,...,...
4921,248-880-0123,2.488800e+09
5346,517-205-7060,NaN
5358,800-436-7936,2.486669e+09
5459,586-726-0044,5.867260e+09


In [41]:
for col in matched_henry.columns:
    print(col)

BIRTH_CITY
BIRTH_COUNTRY
BIRTH_STATE
BIRTH_YEAR
CITY
CITY_MATCH
DEPARTMENT
DESC_PRIM
DESC_SEC
FAX
FAX_NUMBER
FIRST_NAME
GME_INST_ID
GME_INST_STATE
GME_SPEC_1
GME_SPEC_2
LAST_NAME
LOCATION_ADDRESS
LOCATION_ADDRESS_CITY
LOCATION_ADDRESS_STATE
LOCATION_ADDRESS_ZIP
LOCATION_NAME
MAILING_LINE_1
MAILING_LINE_2
MAILING_NAME
MD_DO_CODE
ME
MEDSCHOOL_GRAD_YEAR
MEDSCHOOL_ID
MEDSCHOOL_STATE
MIDDLE_NAME_x
MIDDLE_NAME_y
MOST_RECENT_FORMER_FIRST_NAME
MOST_RECENT_FORMER_LAST_NAME
MOST_RECENT_FORMER_MIDDLE_NAME
NAME
NEXT_MOST_RECENT_FORMER_FIRST
NEXT_MOST_RECENT_FORMER_LAST
NEXT_MOST_RECENT_FORMER_MIDDLE
NICKNAME
PARENT_ORG_NAME
PHONE
POLO_CITY
POLO_MAILING_LINE_1
POLO_MAILING_LINE_2
POLO_STATE
POLO_ZIP
POST_GRADUATE_YEAR
PRIM_SPEC_CD
PROGRAM
SECONDARY_SPECIALTY
SEC_SPEC_CD
SPECIALTY
SPEC_CD_PRIM
SPEC_CD_SEC
SPEC_MATCH
STATE
STATE_MATCH
SUFFIX_x
SUFFIX_y
TELEPHONE_NUMBER
TITLE
TOP_CD
TRAINING_TYPE
YEAR_IN_PROGRAM
ZIP


In [45]:
matched_henry.dropna(subset=['ME'])

,BIRTH_CITY,BIRTH_COUNTRY,BIRTH_STATE,BIRTH_YEAR,CITY,CITY_MATCH,DEPARTMENT,DESC_PRIM,DESC_SEC,FAX,...,STATE,STATE_MATCH,SUFFIX_x,SUFFIX_y,TELEPHONE_NUMBER,TITLE,TOP_CD,TRAINING_TYPE,YEAR_IN_PROGRAM,ZIP
0,NaN,NaN,NaN,1976.0,DEARBORN HTS,NaN,None,CLINICAL NEUROPHYSIOLOGY,NEUROMUSCULAR MEDICINE (NEUROLOGY),None,...,MI,NaN,NaN,,3.139163e+09,,20.0,NaN,NaN,48127
139,DETROIT,US1,MI,1958.0,WASHINGTON,NaN,None,ANESTHESIOLOGY,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,5.862632e+09,,20.0,NaN,NaN,48094
173,DETROIT,US1,MI,1968.0,ROCHESTER HLS,NaN,None,FAMILY MEDICINE,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,5.867265e+09,,20.0,NaN,NaN,48309
214,HYDERABAD,IN1,NaN,1967.0,JACKSON,NaN,None,PHYSICAL MEDICINE AND REHABILITATION,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,5.172051e+09,,20.0,NaN,NaN,49201
215,MAYFIELD HEIGHT,US1,OH,1981.0,ANN ARBOR,NaN,None,PAIN MEDICINE,PAIN MANAGEMENT,None,...,MI,NaN,NaN,,7.349367e+09,,20.0,NaN,NaN,48108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4921,NaN,IZ1,NaN,1980.0,BLOOMFLD HLS,NaN,None,FAMILY MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,2.488800e+09,,20.0,NaN,NaN,48302
5346,STAUNTON,US1,VA,1972.0,JACKSON,NaN,None,INTERNAL MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,NaN,,20.0,NaN,NaN,49201
5358,ADRIAN,US1,MI,1951.0,WATERFORD,NaN,None,INTERNAL MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,2.486669e+09,,20.0,NaN,NaN,48327
5459,NaN,IN1,NaN,1964.0,STERLING HTS,NaN,None,INTERNAL MEDICINE,UNSPECIFIED,None,...,MI,True,NaN,,5.867260e+09,,20.0,NaN,NaN,48313


In [46]:
henry

,NAME,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME
0,Saadia Abbas,Family Medicine,None,586-421-3052,None,Henry Ford Medical Center Sterling Heights,3500 15 Mile Rd,Sterling Heights,MI,48310,None,None,Henry Ford Health System
1,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Bruce Twp,80650 Van Dyke,Bruce Township,MI,48065,None,None,Henry Ford Health System
2,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Chesterfield,30795 23 Mile Rd,Chesterfield,MI,48047,None,None,Henry Ford Health System
3,Sami Abbasi,Dermatology,None,734-675-0835,None,None,"133 W Main St, Ste 251",Northville,MI,48167,None,None,Henry Ford Health System
4,Sami Abbasi,Dermatology,None,734-675-0835,None,Abbasi Dermatology,21401 Allen Road,Woodhaven,MI,48183,None,None,Henry Ford Health System
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7128,Dace Zvirbulis,Neurology,None,800-436-7936,None,Henry Ford Medical Center - Farmington Road,6530 Farmington Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System
7129,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System
7130,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford West Bloomfield Hospital,6777 W Maple Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System
7131,Bryan Zweig,Cardiology,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System


In [58]:
mathced_test = matched_henry[['NAME','ME','TELEPHONE_NUMBER','PHONE']]

In [83]:
pd.merge(henry, SIMPLE, on='NAME', how = 'left')

,NAME,SPECIALTY,SECONDARY_SPECIALTY,PHONE,FAX,LOCATION_NAME,LOCATION_ADDRESS,LOCATION_ADDRESS_CITY,LOCATION_ADDRESS_STATE,LOCATION_ADDRESS_ZIP,DEPARTMENT,PROGRAM,PARENT_ORG_NAME,ME
0,Saadia Abbas,Family Medicine,None,586-421-3052,None,Henry Ford Medical Center Sterling Heights,3500 15 Mile Rd,Sterling Heights,MI,48310,None,None,Henry Ford Health System,7.040297e+10
1,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Bruce Twp,80650 Van Dyke,Bruce Township,MI,48065,None,None,Henry Ford Health System,7.040297e+10
2,Saadia Abbas,Family Medicine,None,586-421-3052,None,HFM Urgent Care - Chesterfield,30795 23 Mile Rd,Chesterfield,MI,48047,None,None,Henry Ford Health System,7.040297e+10
3,Sami Abbasi,Dermatology,None,734-675-0835,None,None,"133 W Main St, Ste 251",Northville,MI,48167,None,None,Henry Ford Health System,2.576010e+09
4,Sami Abbasi,Dermatology,None,734-675-0835,None,Abbasi Dermatology,21401 Allen Road,Woodhaven,MI,48183,None,None,Henry Ford Health System,2.576010e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7128,Dace Zvirbulis,Neurology,None,800-436-7936,None,Henry Ford Medical Center - Farmington Road,6530 Farmington Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System,9.131694e+10
7129,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,2.501712e+09
7130,Raimonds Zvirbulis,Orthopedic Surgery,None,800-436-7936,None,Henry Ford West Bloomfield Hospital,6777 W Maple Rd,West Bloomfield,MI,48322,None,None,Henry Ford Health System,2.501712e+09
7131,Bryan Zweig,Cardiology,None,800-436-7936,None,Henry Ford Medical Center - Fairlane,19401 Hubbard Dr,Dearborn,MI,48126,None,None,Henry Ford Health System,1.720042e+09


,NAME,ME,TELEPHONE_NUMBER,PHONE
0,Youssef Dakka,6.610204e+10,3.139163e+09,800-436-7936
139,Robert Peleman,2.507842e+09,5.862632e+09,586-263-2371
173,Theodore Tangalos,2.507942e+09,5.867265e+09,586-726-4823
214,Anandeep Kumar,4.957091e+10,5.172051e+09,517-205-7525
215,Sarah Money,3.843081e+09,7.349367e+09,800-436-7936
...,...,...,...,...
4921,Fadi Ibrahim,3.050108e+10,2.488800e+09,248-880-0123
5346,Richard Santos,1.803980e+09,NaN,517-205-7060
5358,Christine Meyer,2.501831e+09,2.486669e+09,800-436-7936
5459,William Massey,4.950890e+10,5.867260e+09,586-726-0044


In [81]:
new_phone_count =0
no_count =0
yes_count =0
for row in mathced_test.itertuples():
    if np.isnan(row.TELEPHONE_NUMBER)==False:
        if str(int(row.TELEPHONE_NUMBER)) == str(row.PHONE).replace('-',''):
            yes_count +=1
        else:
            no_count += 1
    elif row.PHONE != 'None':
        new_phone_count += 1
    else:
        print(row.PHONE)
        print(row.TELEPHONE_NUMBER)
print(f'{yes_count} phones match. {no_count} phones do not match. {new_phone_count} physicians have no ppd phone number, but do have a number on the online roster')

None
nan
None
nan
None
nan
674 phones match. 1305 phones do not match. 944 physicians have no ppd phone number, but do have a number on the online roster


In [70]:
import numpy as np

In [76]:
new_phone_count

944

In [79]:
674+1305+944

2923

In [82]:
mathced_test

,NAME,ME,TELEPHONE_NUMBER,PHONE
0,Youssef Dakka,6.610204e+10,3.139163e+09,800-436-7936
139,Robert Peleman,2.507842e+09,5.862632e+09,586-263-2371
173,Theodore Tangalos,2.507942e+09,5.867265e+09,586-726-4823
214,Anandeep Kumar,4.957091e+10,5.172051e+09,517-205-7525
215,Sarah Money,3.843081e+09,7.349367e+09,800-436-7936
...,...,...,...,...
4921,Fadi Ibrahim,3.050108e+10,2.488800e+09,248-880-0123
5346,Richard Santos,1.803980e+09,NaN,517-205-7060
5358,Christine Meyer,2.501831e+09,2.486669e+09,800-436-7936
5459,William Massey,4.950890e+10,5.867260e+09,586-726-0044


In [14]:
missing_df = brig_df[brig_df.NAME.isin(missing_names_brig)]

In [29]:
ppd[ppd['LAST_NAME']=='ABOU RJEILY']

,SPEC_CD_SEC,DESC_SEC,SPEC_CD_PRIM,DESC_PRIM,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,...,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
649576,US,UNSPECIFIED,HOS,HOSPITALIST,60503040240,CHARBEL M ABOU RJEILY MD,ABOU RJEILY,CHARBEL,MELHEM DIT YOUSSEF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
ppd[ppd['FIRST_NAME']=='NEZAM']

,SPEC_CD_SEC,DESC_SEC,SPEC_CD_PRIM,DESC_PRIM,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,...,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
580265,US,UNSPECIFIED,GE,GASTROENTEROLOGY,53906810138,NEZAM H AFDAHL MD,AFDAHL,NEZAM,H,NaN,...,110 FRANCIS ST STE GB,BOSTON,MA,2215,AFDHAL,H,NEZAM,NaN,NaN,NaN
1169384,US,UNSPECIFIED,RHU,RHEUMATOLOGY,56001020019,NEZAM ALTOROK MD,ALTOROK,NEZAM,NaN,NaN,...,3125 TRANSVERSE DR,TOLEDO,OH,43614,TOROK,NaN,NEZAM,NaN,NaN,NaN


In [41]:
ppd[ppd['LAST_NAME']=='ALLENDICKER']

,SPEC_CD_SEC,DESC_SEC,SPEC_CD_PRIM,DESC_PRIM,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,...,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
715148,US,UNSPECIFIED,IM,INTERNAL MEDICINE,3519091363,JOSHUA W ALLENDICKER MD,ALLENDICKER,JOSHUA,WILL,NaN,...,330 BROOKLINE AVE,BOSTON,MA,2215.0,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
matched_brig.drop_duplicates()[['MAILING_NAME','NAME','DESC_PRIM','SPECIALTY','CITY','POLO_CITY']]

,MAILING_NAME,NAME,DESC_PRIM,SPECIALTY,CITY,POLO_CITY
0,RACHEL A NARDIN MD,"Rachel Austen Nardin, MD",NEUROLOGY,Neurology,CAMBRIDGE,CAMBRIDGE
68,PUSHPA NARAYANASWAMI MD,"Pushpa Narayanaswami, MD",NEUROLOGY,Neurology,BOSTON,BOSTON
69,ROSS J MUSUMECI MD,"Ross J. Musumeci, MD",ANESTHESIOLOGY,Anesthesiology,LOWELL,LOWELL
114,ANDREW J KOROPEY MD,"Andrew J. Koropey, MD",ANESTHESIOLOGY,Anesthesiology,COHASSET,PROVIDENCE
124,RICHARD P SHOCKLEY MD,"Richard Paul Shockley, MD",ANESTHESIOLOGY,Anesthesiology,WELLESLEY,WESTWOOD
...,...,...,...,...,...,...
5272,MARY E MANNING MD,"Mary E. Manning, MD, MPH",INTERNAL MEDICINE,Internal Medicine,NEEDHAM,NEEDHAM
4044,KEVIN M HILL MD,"Kevin P. Hill, MD, MHS",PSYCHIATRY,Psychiatry,SARASOTA,BOSTON
4040,KEVIN P HILL MD,"Kevin P. Hill, MD, MHS",ADDICTION PSYCHIATRY,Psychiatry,NEWTON CENTRE,BOSTON
4240,AN K PHAN MD,"An K. Phan, MD",FAMILY MEDICINE,Family Medicine,QUINCY,DORCHESTER


Index(['BIRTH_CITY', 'BIRTH_COUNTRY', 'BIRTH_STATE', 'BIRTH_YEAR', 'CITY',
       'CITY_MATCH', 'DEPARTMENT', 'DESC_PRIM', 'DESC_SEC', 'FAX',
       'FAX_NUMBER', 'FIRST_NAME', 'GME_INST_ID', 'GME_INST_STATE',
       'GME_SPEC_1', 'GME_SPEC_2', 'LAST_NAME', 'LOCATION_ADDRESS',
       'LOCATION_ADDRESS_CITY', 'LOCATION_ADDRESS_STATE',
       'LOCATION_ADDRESS_ZIP', 'LOCATION_NAME', 'LOCATION_STREET_ADDRESS',
       'MAILING_LINE_1', 'MAILING_LINE_2', 'MAILING_NAME', 'MD_DO_CODE', 'ME',
       'MEDSCHOOL_GRAD_YEAR', 'MEDSCHOOL_ID', 'MEDSCHOOL_STATE',
       'MIDDLE_NAME_x', 'MIDDLE_NAME_y', 'MOST_RECENT_FORMER_FIRST_NAME',
       'MOST_RECENT_FORMER_LAST_NAME', 'MOST_RECENT_FORMER_MIDDLE_NAME',
       'NAME', 'NEXT_MOST_RECENT_FORMER_FIRST', 'NEXT_MOST_RECENT_FORMER_LAST',
       'NEXT_MOST_RECENT_FORMER_MIDDLE', 'NICKNAME', 'PARENT_ORG_NAME',
       'PHONE', 'POLO_CITY', 'POLO_MAILING_LINE_1', 'POLO_MAILING_LINE_2',
       'POLO_STATE', 'POLO_ZIP', 'POST_GRADUATE_YEAR', 'PRIM_SPEC_CD',


In [47]:
ppd[(ppd['FIRST_NAME']=='KEVIN')&(ppd['LAST_NAME']=='HILL')]

,SPEC_CD_SEC,DESC_SEC,SPEC_CD_PRIM,DESC_PRIM,ME,MAILING_NAME,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,...,POLO_MAILING_LINE_2,POLO_CITY,POLO_STATE,POLO_ZIP,MOST_RECENT_FORMER_LAST_NAME,MOST_RECENT_FORMER_MIDDLE_NAME,MOST_RECENT_FORMER_FIRST_NAME,NEXT_MOST_RECENT_FORMER_LAST,NEXT_MOST_RECENT_FORMER_MIDDLE,NEXT_MOST_RECENT_FORMER_FIRST
238843,US,UNSPECIFIED,ADP,ADDICTION PSYCHIATRY,4102011240,KEVIN P HILL MD,HILL,KEVIN,PATRICK,NaN,...,330 BROOKLINE AVE BLDG GRZ133,BOSTON,MA,2215.0,NaN,NaN,NaN,NaN,NaN,NaN
371194,US,UNSPECIFIED,DR,DIAGNOSTIC RADIOLOGY,3503890461,KEVIN F HILL MD,HILL,KEVIN,FRANCIS,NaN,...,1421 LEXINGTON AVE,MANSFIELD,OH,44907,NaN,NaN,NaN,NaN,NaN,NaN
575084,US,UNSPECIFIED,GE,GASTROENTEROLOGY,3575961494,KEVIN B HILL DO,HILL,KEVIN,B,NaN,...,300 SIERRA COLLEGE DR STE 105,GRASS VALLEY,CA,95945,HILL,BOYD,KEVIN,NaN,NaN,NaN
724350,US,UNSPECIFIED,IM,INTERNAL MEDICINE,3843952762,KEVIN K HILL MD,HILL,KEVIN,KENNETH,NaN,...,2600 TUSCARAWAS ST W STE 440,CANTON,OH,44708,NaN,NaN,NaN,NaN,NaN,NaN
980066,US,UNSPECIFIED,P,PSYCHIATRY,1105191100,KEVIN M HILL MD,HILL,KEVIN,MATTHEW,NaN,...,330 BROOKLINE AVE,BOSTON,MA,2215.0,NaN,NaN,NaN,NaN,NaN,NaN
1103325,US,UNSPECIFIED,PDC,PEDIATRIC CARDIOLOGY,3605020916,KEVIN D HILL MD,HILL,KEVIN,DENNIS,NaN,...,1991 FORDHAM DR STE 200,FAYETTEVILLE,NC,28304,NaN,NaN,NaN,NaN,NaN,NaN
1123765,US,UNSPECIFIED,PM,PHYSICAL MEDICINE AND REHABILITATION,3005870509,KEVIN S HILL MD,HILL,KEVIN,SCOTT,NaN,...,777 HOSPITAL WAY STE G15,POCATELLO,ID,83201,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
new_duplicates_3[['MIDDLE_NAME_x','MIDDLE_NAME_y']]

,MIDDLE_NAME_x,MIDDLE_NAME_y
4040,PATRICK,P.
4044,MATTHEW,P.
4239,THOAI,K.
4240,KIEU,K.
6323,M,P.
6324,PAUL,P.


In [1]:
def find_missing_1(missing_df,ppd):
    dict_list_1 =[]
    for row in missing_df.itertuples():
        print(row.LAST_NAME)
        new_df = ppd[ppd.LAST_NAME == row.LAST_NAME]
        for line in new_df.itertuples():
            match =False
            if fuzz.ratio(row.FIRST_NAME, line.FIRST_NAME)>85:
                match = True
            elif row.FIRST_NAME in line.FIRST_NAME:
                match = True
            elif row.MIDDLE_NAME == line.FIRST_NAME:
                match = True
            elif len(line.MIDDLE_NAME) > 4 and line.MIDDLE_NAME in row.FIRST_NAME:
                match = True
            elif row.FIRST_NAME+' '+row.FIRST_NAME == line.FIRST_NAME:
                match = True
            elif row.FIRST_NAME.replace('.','')==line.FIRST_NAME[0]:
                match = True
            elif row.FIRST_NAME[0]==line.FIRST_NAME:
                match = True
            if match == True:
                new_dict = {
                    'NAME':row.NAME,
                    'ME':line.ME
                }
                dict_list_1.append(new_dict)
    return(dict_list_1)

def find_missing_2(missing_df,ppd):
    dict_list_1 =[]
    for row in missing_df.itertuples():
        print(row.LAST_NAME)
        match = False
        new_df = ppd[ppd.FIRST_NAME == row.FIRST_NAME]
        print(len(new_df))
        for line in new_df.itertuples():
            match = False
            if row.LAST_NAME in line.LAST_NAME:
                print(f'{row.NAME} matches {line.MAILING_NAME}')
                match = True
            elif len(line.LAST_NAME)>4 and line.LAST_NAME in row.LAST_NAME:
                print(f'{row.NAME} matches {line.MAILING_NAME}')
                match = True
            elif fuzz.ratio(row.LAST_NAME, line.LAST_NAME)>83:
                print(f'{row.NAME} matches {line.MAILING_NAME}')
                match = True
            elif row.MIDDLE_NAME == line.LAST_NAME:
                print(f'{row.NAME} matches {line.MAILING_NAME}')
                match = True
            elif row.MIDDLE_NAME+row.LAST_NAME == line.LAST_NAME:
                print(f'{row.NAME} matches {line.MAILING_NAME}')
                match = True
            elif row.SUFFIX.split(',')[0] == line.LAST_NAME:
                print(f'{row.NAME} matches {line.MAILING_NAME}')
                match = True
            if match == True:
                new_dict = {
                    'NAME':row.NAME,
                    'ME':line.ME
                }
                dict_list_1.append(new_dict)
    return(dict_list_1)

def clean_missing_matches(missing_match_df):
    names ={}
    for row in missing_match_df.itertuples():
        keep = False
        if row.STATE == row.POLO_STATE:
            if (row.MD_DO_CODE == 1 and 'MD' in row.SUFFIX_y) or (row.MD_DO_CODE == 2 and 'DO' in row.SUFFIX_y):
                if fuzz.ratio(row.NAME.upper(), row.MAILING_NAME)>72:
                    if fuzz.ratio(row.SPECIALTY.upper(), row.DESC_PRIM) > 80:
                        keep = True
                    elif row.DESC_PRIM in row.SPECIALTY.upper():
                        keep = True
                    elif row.SPECIALTY.upper() in row.DESC_PRIM:
                        keep = True
                    elif row.SPECIALTY.upper() in row.DESC_SEC:
                        keep = True
                    elif fuzz.ratio(row.SPECIALTY.upper(),row.DESC_SEC) > 80:
                        keep = True

                    mid_match = False
                    if row.MIDDLE_NAME_y != 'None' :
                        print(row.MIDDLE_NAME_y)
                        print(row.MIDDLE_NAME_x)
                        if row.MIDDLE_NAME_x == row.MIDDLE_NAME_y:
                            mid_match = True
                        elif row.MIDDLE_NAME_y:
                            if row.MIDDLE_NAME_x[0] == row.MIDDLE_NAME_y[0]:
                                mid_match = True
                    if keep == True:
                        if row.NAME in names.keys():
                            if names[row.NAME][1]==False and mid_match ==True:
                                names[row.NAME][0]=row.ME
                                names[row.NAME][1]=mid_match
                        if row.NAME not in names.keys():
                            names[row.NAME] =[row.ME,mid_match,row.NAME]
    return(names)

In [17]:
missing_df = missing_df.fillna('None')
ppd = ppd.fillna('None')

In [19]:
name_dict_list = find_missing_1(missing_df, ppd)

RJEILY
AFDHAL
AHN
ALLEN-DICKER
ANAGNOSTOPOULOS
MOLINS
BIENIA-KENTON
BOCKORNY
GREALLY
BREWER
BYSE
CATANA
CHABOT
CHEE
D.
CIBOTTI-GRANOF
COLLIER
CRAFORD
CRITCHLOW
POLANCO
CRUZ-ROMERO
CULIC
DARVISH
DE-MATTEO
DENO
DINH
V.
DONLAGIC
DONNELLY
DORVIL
AKIKI
ZOGHBI
ELGEZIRY
ELIOPOULOS
ESTEGHAMATI
FALCHUK
FAN
FEIBISH
FERZOCO
FIELD
FILIN
FINE
FORROW
IBANEZ
GANI
GARCIA-WEBB
STOCKTON
GAVRILA-FILIP
GELFAND
GIURINI
GLEYSTEEN
GOLEN
HARTIGAN
HAY
HEHER
HESS
HOCHSTIN
HURWITZ
MOREDA
KANJEE-KHOJA
KATO
KIM
KOHLI
KOZIEL
KRIEGEL
KU
KUNDI-SHARMA
LAMONT
LEEMAN
LIM
LONIS-SCHEUB
LUPTAKOVA
MUKHERJEE
MCALMON
MCARDLE
MCGIRR
MCPHILLIPS
MENDES
MOCZYNSKI
MORAN
NGHIEM
NGUYEN
MHUIRCHEARTAIGH
NORIAN
NYMAN
OLSEN
OLVECZKY
OREN-GRINBERG
OTT
PECK
PIRYATINSKY
POLAN
PRO-RISQUEZ
RAMACHANDRAN
RAMOS-AYES
RASHID
RIEMER
ROCK
RODRIGUE
ROJAS
RYAN-BARNETT
SAYEED-SHAH
SCHAEFER
SEARLS
SENTHILNATHAN
SHARKEY
SHAUGHNESSY
SHERMAN
SHLAIN-FRINK
SHOOK
TESS
TVETER
CAMACHO
VANDERHORST
VANDERLAAN
VEMULA
VERMA
WAYLER
WEISS
WILCOX
WOOTTON
WYLIE
YAZDI


In [20]:
name_dict_list_2 = find_missing_2(missing_df, ppd)

RJEILY
15
Charbel M. Abou Rjeily, MD matches CHARBEL M ABOU RJEILY MD
AFDHAL
2
AHN
270
ALLEN-DICKER
2940
Joshua W. Allen-Dicker, MD matches JOSHUA A ALLEN MD
Joshua W. Allen-Dicker, MD matches JOSHUA ALLEN DO
Joshua W. Allen-Dicker, MD matches JOSHUA M ALLEN MD
Joshua W. Allen-Dicker, MD matches JOSHUA W ALLENDICKER MD
Joshua W. Allen-Dicker, MD matches JOSHUA D ALLEN MD
ANAGNOSTOPOULOS
49
MOLINS
126
Joaquin Bellmunt Molins, MD matches JOAQUIN BELLMUNT MOLINS MD
BIENIA-KENTON
3525
Lisa M. Bienia-Kenton, MD matches LISA M BIENIA MD
BOCKORNY
92
Bruno B. Bockorny, MD matches BRUNO BOMFIM DA S BOCKORNY MD
GREALLY
496
Stacey Breen Greally, MD matches STACEY M BREEN MD
BREWER
298
BYSE
2173
CATANA
0
CHABOT
871
CHEE
78
D.
3039
Frank D. Chi, MD matches FRANK D CHI MD
CIBOTTI-GRANOF
2212
Nancy Cibotti-Granof, MD matches NANCY C GRANOF MD
COLLIER
0
CRAFORD
27211
John Paul E. Craford, MD matches JOHN L CRAWFORD JR MD
John Paul E. Craford, MD matches JOHN J CRAWFORD MD
John Paul E. Craford, MD matc

ROCK
3556
Laura Kane Rock, MD matches LAURA H ROCKER MD
Laura Kane Rock, MD matches LAURA C THROCKMORTON MD
Laura Kane Rock, MD matches LAURA B KANE MD
Laura Kane Rock, MD matches LAURA M CROCKER MD
RODRIGUE
20384
James R. Rodrigue, PhD matches JAMES J RODRIGUEZ MD
James R. Rodrigue, PhD matches JAMES E RODRIGUEZ MD
James R. Rodrigue, PhD matches JAMES LUGO RODRIGUEZ MD
James R. Rodrigue, PhD matches JAMES J RODRIGUES MD
James R. Rodrigue, PhD matches JAMES RODRIGUEZ MD
James R. Rodrigue, PhD matches JAMES W RODRIGUEZ DO
ROJAS
886
Rafael Rojas, MD matches RAFAEL G ROJAS ROMAN MD
Rafael Rojas, MD matches RAFAEL ROJAS JASSO MD
RYAN-BARNETT
536
Sheila M. Ryan-Barnett, MD matches SHEILA R BARNETT MD
SAYEED-SHAH
27
SCHAEFER
36
SEARLS
24874
David Eric C. Searls, MD matches DAVID A SEARS MD
David Eric C. Searls, MD matches DAVID A SEARS MD
David Eric C. Searls, MD matches DAVID R SEARS MD
David Eric C. Searls, MD matches DAVID E SEALS MD
SENTHILNATHAN
4
SHARKEY
22
SHAUGHNESSY
868
SHERMAN
282


MCKEON
6405
Brian P. McKeon, MD matches BRIAN P MC KEON MD
MCKINNEY
538
Renee M. McKinney, MD matches RENEE M MC KINNEY MD
MCSHINE
977
Randall H. McShine, MD matches RANDALL H MC SHINE MD
MCSWEENEY
4615
Gregory M. McSweeney, MD matches GREGORY D SWEENEY MD
Gregory M. McSweeney, MD matches GREGORY C SWEENEY MD
Gregory M. McSweeney, MD matches GREGORY M MC SWEENEY MD
MEHRA
7
RIOS
4
Nyssa Z. Melendez Rios, MD matches NYSSA Z MELENDEZ-RIOS MD
MEYDANI-KORB
1
Ahou Meydani-Korb, MD matches AHOU MEYDANI MD
GHOMIZADEH
99
Shadi Miri Ghomizadeh, MD matches SHADI MIRI GHOMIZADEH MD
MORRISON
187
MOUGIANIS
0
MOUSSA-PERVANE
1
Runna F. Moussa-Pervane, MD matches RUNNA F MOUSSA MD
NGHIEM
669
OBASIOLU
0
O'CONNOR
1820
O'KEEFE
2135
OKPOKWASILI-JOHNSON
8
Ebele Chiagoziem Okpokwasili-Johnson, MD matches EBELE C OKPOKWASILI MD
OLIVER
3
OLIVET
579
Hannah L. Olivet, MD matches HANNAH L BOULDINOLIVET MD
OZA
4
Sveta Shah Oza, MD matches SVETA C SHAH MD
PAGANO
8650
PAPPAVASELIO
12150
PATEL
1
Apranta Deka Patel, M

In [22]:
missing_matches = pd.DataFrame(name_dict_list + name_dict_list_2)



In [25]:
missing_matches = pd.merge(brig_df, missing_matches, on = 'NAME')
missing_matches = pd.merge(ppd, missing_matches, on = 'ME')
missing_matches = missing_matches.drop_duplicates(['NAME','ME'])

pd.DataFrame(clean_missing_matches(missing_matches)).transpose()

AttributeError: 'Pandas' object has no attribute 'STATE'